# Analyzing the performance of models on the same dataset

## Mount Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pwd

/content


In [4]:
%cd "drive/MyDrive/22-23/DS 301 Advanced Data Science/DS 301 Final Project"

/content/drive/.shortcut-targets-by-id/1L_EikC2ryU5sSu8xLqkBA2WHnRs7BMt-/DS 301 Final Project


In [5]:
!pwd

/content/drive/.shortcut-targets-by-id/1L_EikC2ryU5sSu8xLqkBA2WHnRs7BMt-/DS 301 Final Project


In [6]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Import

In [7]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57.4 MB 1.3 MB/s 
     |████████████████████████████████| 8.8 MB 25.6 MB/s 
     |████████████████████████████████| 468 kB 55.5 MB/s 


In [8]:
# importing the libraries

import pandas as pd
import numpy as np
import copy

# for visualization

import plotly.express as px
import matplotlib.pyplot as plt

# hypeparameter tuning

import ray
from ray import tune
from ray.tune.schedulers import HyperBandScheduler

In [9]:
# models

import statsmodels.api as sm
import sklearn.model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import xgboost

In [10]:
# dict used to convert state to abbrev

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}


## Reading data/helper functions

In [11]:
# Reading the training data

data = pd.read_csv('state_data_2019_processed_training.csv')

states = data.sitecode.unique()

In [12]:
# Creating the training/testing sets

state_data_t = []

state_data_test = []

seed = 100

for state in states:

    state_data = data[data.sitecode == state]
    X_s = state_data.iloc[:,:-1]
    y_s = state_data.iloc[:,-1]
    X_tr, X_val, y_tr, y_val = sklearn.model_selection.train_test_split(
        X_s, y_s, test_size=0.4, random_state=seed, stratify=y_s)
    X_val, X_te, y_val, y_te = sklearn.model_selection.train_test_split(
        X_val, y_val, test_size=0.5, random_state=seed, stratify=y_val)
    state_data_t.append([X_tr, X_val, y_tr, y_val])
    state_data_test.append((X_te, y_te))

X_train, X_val, y_train, y_val = state_data_t[0]

for i in range(1, len(state_data_t)):
    X_t,X_v,y_t,y_v = state_data_t[i]
    X_train = pd.concat([X_train, X_t])
    y_train = pd.concat([y_train, y_t])
    X_val = pd.concat([X_val, X_v])
    y_val = pd.concat([y_val, y_v])

In [13]:
# functions for visualizing the results

def map(model, race = 'all', exclude = None, data = state_data_t):

    auc_all = pd.DataFrame({'state': [us_state_to_abbrev[s] for s in states], 'auc': np.zeros(len(states))})

    for i, state in enumerate(states):
        _, X_test, _, y_test = data[i]
        if(race != 'all'):
            if(race == 'black'):
                index = X_test['race4_Black or African American'] == 1
            if(race == 'hispanic'):
                index = X_test['race4_Hispanic/Latino'] == 1
            X_test = X_test.loc[index]
            y_test = y_test.loc[index]
        if(exclude != None):
            X_test = X_test.drop(columns = exclude)
        y_pred = model.predict_proba(X_test.iloc[:,4:])
        fpr, tpr,_ = metrics.roc_curve(y_test, y_pred[:,1])
        auc = metrics.auc(fpr,tpr)
        auc_all.iloc[i,1] = auc

    fig = px.choropleth(auc_all,
                        locations='state', 
                        locationmode="USA-states", 
                        scope="usa",
                        color='auc',
                        color_continuous_scale="Viridis_r",
                        range_color = [0.6,1] 
                        )
    return fig, auc_all

def count_race(data = state_data_t):
    race = pd.DataFrame({
        'others': np.zeros(len(states)),
        'black': np.zeros(len(states)),
        'hispanic': np.zeros(len(states)),
        'white': np.zeros(len(states)),
        'missing': np.zeros(len(states))
    }, index = states)
    for i, state in enumerate(states):
        _, X_test, _, _ = data[i]
        race.iloc[i,0] = X_test['race4_All other races'].sum()
        race.iloc[i,1] = X_test['race4_Black or African American'].sum()
        race.iloc[i,2] = X_test['race4_Hispanic/Latino'].sum()
        race.iloc[i,3] = X_test['race4_White'].sum()
        race.iloc[i,4] = X_test['race4_All other races'].count() - \
            race.iloc[i,0] - race.iloc[i,1] -race.iloc[i,2] -race.iloc[i,3]
    
    return race

## Logisitc Model

In [14]:
logistic_model = LogisticRegression(
    penalty='l2',
    max_iter=200
)
logistic_model = logistic_model.fit(X_train.iloc[:,4:], y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
logistic_model_map, logistic_auc = map(logistic_model)

In [16]:
logistic_model_map.show()

## Random Forest Model

In [17]:
!pip install tune_sklearn
!pip install hpbandster ConfigSpace

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 114 kB/s 
     |████████████████████████████████| 125 kB 14.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 110 kB/s 
     |████████████████████████████████| 5.7 MB 16.3 MB/s 
     |████████████████████████████████| 89 kB 11.7 MB/s 
  Created wheel for hpbandster: filename=hpbandster-0.7.4-py3-none-any.whl size=80008 sha256=fa8a8ff7f2d81b1cebf9136d320f9df85a49a6c89586f4407f8594838e649bba
  Stored in directory: /root/.cache/pip/wheels/f6/42/dd/82bd75284b37e199206b84c6c5a057499687e6fa530674d43e
Successfully built hpbandster


In [18]:
# Hyperparameter tuning using grid search

from tune_sklearn import TuneSearchCV
from tune_sklearn import TuneGridSearchCV

params = {
    'max_depth': [10,20,30,40],
    'min_samples_leaf' : [5,10,15,20],
    'min_samples_split': [10,20,30,40],
    'max_features' : [40,80,120,160],
}

In [ ]:
tune_grid_search = TuneGridSearchCV(
    RandomForestClassifier(
        n_estimators = 200,
        random_state = seed, 
        criterion = 'gini'
        ),
    params,
    scoring = 'roc_auc',
    verbose=2,
    n_jobs = -1,
    use_gpu=True,
    early_stopping="MedianStoppingRule",
    max_iters=10
)

result_grid = tune_grid_search.fit(X_train.iloc[:,4:], y_train)

In [ ]:
tune_search = TuneSearchCV(
    RandomForestClassifier(
        n_estimators = 200,
        random_state = seed, 
        criterion = 'gini'
        ),
    params,
    scoring = 'roc_auc',
    verbose=2,
    n_jobs = -1,
    early_stopping="MedianStoppingRule",
    n_trials=20,
    max_iters=10,
    search_optimization="bohb"
)

result = tune_search.fit(X_train.iloc[:,4:], y_train)

In [ ]:
tune_grid_search.best_params

In [ ]:
tune_search.best_params

In [ ]:
rf_model = RandomForestClassifier(
    max_depth=30,
    max_features=40, 
    min_samples_leaf=20,
    min_samples_split=10,
    n_estimators=200,
    random_state = seed, 
    criterion = 'gini'
)
rf_model.fit(X_train.iloc[:,4:], y_train)

In [ ]:
rf_model_map, rf_auc = map(rf_model)

In [ ]:
rf_model_map.show()

## XGBoost Model

### Hyperparameter tuning

In [ ]:
import xgboost as xgb
from ray.tune.integration.xgboost import TuneReportCheckpointCallback

def data_loader():
    return (X_train.iloc[:,4:].values,y_train.values), (X_val.iloc[:,4:].values,y_val.values)
def train_data(config,data):
    t1, t2 = data
    train_set = xgb.DMatrix(t1[0], label = t1[1])
    val_set = xgb.DMatrix(t2[0], label = t2[1])
    results = {}
    bst = xgb.train(
        config,
        train_set,
        num_boost_round = 50,
        evals = [(val_set, 'eval')],
        evals_result = results,
        verbose_eval = False,
        callbacks=[TuneReportCheckpointCallback(filename="model.xgb")]
    )

config = {
          "objective": "binary:logistic",
          "tree_method": "gpu_hist",
          "eval_metric": ["auc"],
          "max_depth": tune.randint(5,12),
          "min_child_weight": tune.randint(1,5),
          "colsample_bytree": tune.uniform(0.5, 1.0),
          "eta": tune.loguniform(1e-3, 1e-1),
          "reg_lambda": tune.uniform(0.1, 5),
          "reg_alpha": tune.uniform(0.1, 5),
          "n_estimators" : 200,
          "seed": 0
}

In [ ]:
t1,t2 = data_loader()

analysis = tune.run(
    tune.with_parameters(train_data, data = (t1,t2)),
    resources_per_trial = {"gpu":1},
    config = config,
    num_samples = 50,
    metric='eval-auc',
    mode="max",
    stop={
        "eval-auc": 0.90,
        "training_iteration": 50
    },
)

Trial name,date,done,episodes_total,eval-auc,experiment_id,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_data_76bc1_00000,2022-12-17_18-56-21,True,,0.863921,204ad728ac8542a6a01f836d62fd866c,4a3396a947d5,50,172.28.0.12,7934,6.12926,0.071125,6.12926,1671303381,0,,50,76bc1_00000,0.00364661
train_data_76bc1_00001,2022-12-17_18-56-28,True,,0.859324,1b9b8afe5afd4282806fec38c64db252,4a3396a947d5,50,172.28.0.12,8024,4.34051,0.0462074,4.34051,1671303388,0,,50,76bc1_00001,0.00372529
train_data_76bc1_00002,2022-12-17_18-56-35,True,,0.85591,b5601c9bb20d4c24b0493b2d989ad496,4a3396a947d5,50,172.28.0.12,8079,4.48715,0.0499957,4.48715,1671303395,0,,50,76bc1_00002,0.00404668
train_data_76bc1_00003,2022-12-17_18-56-44,True,,0.85959,365386e03d724535af23a1ffa909dacb,4a3396a947d5,50,172.28.0.12,8139,6.25062,0.0958581,6.25062,1671303404,0,,50,76bc1_00003,0.00448966
train_data_76bc1_00004,2022-12-17_18-56-52,True,,0.847514,d2fa9cb8041548dc9a9aed3e288d6c27,4a3396a947d5,50,172.28.0.12,8193,3.97123,0.0365188,3.97123,1671303412,0,,50,76bc1_00004,0.00480103
train_data_76bc1_00005,2022-12-17_18-56-58,True,,0.851693,f38cf8468571479eb3eff1120fce43f5,4a3396a947d5,50,172.28.0.12,8250,3.11773,0.024745,3.11773,1671303418,0,,50,76bc1_00005,0.00376415
train_data_76bc1_00006,2022-12-17_18-57-05,True,,0.843631,b18f05c52362404b9b3e61148534753b,4a3396a947d5,50,172.28.0.12,8306,4.66637,0.0565474,4.66637,1671303425,0,,50,76bc1_00006,0.00381017
train_data_76bc1_00007,2022-12-17_18-57-11,True,,0.851998,0d815e9293684c04965a800a951b8514,4a3396a947d5,50,172.28.0.12,8361,3.02395,0.0222604,3.02395,1671303431,0,,50,76bc1_00007,0.00384116
train_data_76bc1_00008,2022-12-17_18-57-16,True,,0.828212,30b76c6c5fea4545bf389c6e158ffec2,4a3396a947d5,50,172.28.0.12,8417,2.88305,0.018023,2.88305,1671303436,0,,50,76bc1_00008,0.00352764
train_data_76bc1_00009,2022-12-17_18-57-22,True,,0.860342,c64dbefbd2a1426885984b5031abbbc5,4a3396a947d5,50,172.28.0.12,8470,3.05503,0.0232561,3.05503,1671303442,0,,50,76bc1_00009,0.00419092


2022-12-17 19:01:46,767	INFO tune.py:762 -- Total run time: 339.43 seconds (339.23 seconds for the tuning loop).


In [ ]:
t1,t2 = data_loader()

from ray.tune.schedulers import HyperBandScheduler

hyperband_scheduler = HyperBandScheduler(
    time_attr='training_iteration',
    metric='eval-auc',
    mode='max',
    max_t=50,
    reduction_factor=3)


analysis_hyper = tune.run(
    tune.with_parameters(train_data, data = (t1,t2)),
    resources_per_trial = {"gpu":1},
    config = config,
    num_samples = 50,
    scheduler = hyperband_scheduler
)

Trial name,date,done,episodes_total,eval-auc,experiment_id,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_data_41672_00000,2022-12-17_19-01-52,False,,0.8428,0c41232feb444804b45aba48c8af4ce5,4a3396a947d5,50,172.28.0.12,10774,2.80217,0.0175545,2.80217,1671303712,0,,50,41672_00000,0.00399971
train_data_41672_00001,2022-12-17_19-05-15,False,,0.852407,b5bc3e75254b4dc1af436d08df629629,4a3396a947d5,50,172.28.0.12,12644,3.93599,0.042712,3.93599,1671303915,0,,50,41672_00001,0.00400305
train_data_41672_00002,2022-12-17_19-07-18,False,,0.859902,61acd684d1d34dbea58a70da9a791c3c,4a3396a947d5,50,172.28.0.12,13678,5.26275,0.0722044,5.26275,1671304038,0,,50,41672_00002,0.00460553
train_data_41672_00003,2022-12-17_19-08-10,False,,0.851579,d5596acfc8a549a8bb60809e1f535ff6,4a3396a947d5,50,172.28.0.12,14137,3.64508,0.0329785,3.64508,1671304090,0,,50,41672_00003,0.00462651
train_data_41672_00004,2022-12-17_19-08-40,True,,0.856675,d167e4ccf47c4ed494ea22ba6fb4eeb4,4a3396a947d5,50,172.28.0.12,14443,4.71968,0.0665433,4.71968,1671304120,0,,50,41672_00004,0.0131214
train_data_41672_00005,2022-12-17_19-01-57,False,,0.847959,0e0d1bdc825f4d8492b9f583542845f0,4a3396a947d5,16,172.28.0.12,10827,2.24272,0.023556,2.24272,1671303717,0,,16,41672_00005,0.00420213
train_data_41672_00006,2022-12-17_19-02-22,False,,0.846428,15dbc0c58e5a453191f165583a4b588a,4a3396a947d5,16,172.28.0.12,11056,2.22266,0.0215702,2.22266,1671303742,0,,16,41672_00006,0.00370049
train_data_41672_00007,2022-12-17_19-02-42,False,,0.839631,7214892b6a1f481b873d6ec2862712a2,4a3396a947d5,16,172.28.0.12,11236,2.22741,0.0174448,2.22741,1671303762,0,,16,41672_00007,0.00857306
train_data_41672_00008,2022-12-17_19-05-31,False,,0.840711,970bcd85910b41bba2a0e995ca52d298,4a3396a947d5,16,172.28.0.12,12789,3.13763,0.078742,3.13763,1671303931,0,,16,41672_00008,0.00400949
train_data_41672_00009,2022-12-17_19-08-27,False,,0.852507,420d17d88c99462b8292ffd39fbcb26d,4a3396a947d5,18,172.28.0.12,14279,3.09344,0.0910041,6.20563,1671304107,0,,34,41672_00009,0.0105171


(train_data pid=12373) 2022-12-17 19:04:41,166	INFO trainable.py:790 -- Restored on 172.28.0.12 from checkpoint: /root/ray_results/train_data_2022-12-17_19-01-47/train_data_41672_00028_28_colsample_bytree=0.8709,eta=0.0487,max_depth=9,min_child_weight=4,reg_alpha=0.9045,reg_lambda=4.5838_2022-12-17_19-02-53/checkpoint_tmp9d433e
(train_data pid=12373) 2022-12-17 19:04:41,166	INFO trainable.py:799 -- Current state after restoring: {'_iteration': 0, '_timesteps_total': 0, '_time_total': 2.1411709785461426, '_episodes_total': 0}
2022-12-17 19:04:42,999	INFO hyperband.py:472 -- Restoring from a previous point in time. Previous=1; Now=1
(train_data pid=12418) 2022-12-17 19:04:46,193	INFO trainable.py:790 -- Restored on 172.28.0.12 from checkpoint: /root/ray_results/train_data_2022-12-17_19-01-47/train_data_41672_00029_29_colsample_bytree=0.9988,eta=0.0068,max_depth=10,min_child_weight=1,reg_alpha=3.3517,reg_lambda=2.9621_2022-12-17_19-02-59/checkpoint_tmpfec5fd
(train_data pid=12418) 2022-12

In [ ]:
analysis.get_best_trial(metric = 'eval-auc', mode = 'max').last_result

{'eval-auc': 0.862145,
 'time_this_iter_s': 0.06005525588989258,
 'done': True,
 'timesteps_total': None,
 'episodes_total': None,
 'training_iteration': 50,
 'trial_id': 'c477c_00002',
 'experiment_id': '05ebc83fdd2f4de986c4249a0c524c84',
 'date': '2022-12-17_18-51-43',
 'timestamp': 1671303103,
 'time_total_s': 4.703388214111328,
 'pid': 5508,
 'hostname': '4a3396a947d5',
 'node_ip': '172.28.0.12',
 'config': {'objective': 'binary:logistic',
  'tree_method': 'gpu_hist',
  'eval_metric': ['auc'],
  'max_depth': 10,
  'min_child_weight': 3,
  'colsample_bytree': 0.5020551988135382,
  'eta': 0.029222812929186416,
  'reg_lambda': 0.36756709262675025,
  'reg_alpha': 3.7137701473201687,
  'n_estimators': 200,
  'seed': 0},
 'time_since_restore': 4.703388214111328,
 'timesteps_since_restore': 0,
 'iterations_since_restore': 50,
 'warmup_time': 0.0036313533782958984,
 'experiment_tag': '2_colsample_bytree=0.5021,eta=0.0292,max_depth=10,min_child_weight=3,reg_alpha=3.7138,reg_lambda=0.3676'}

In [ ]:
analysis_hyper.get_best_trial(metric = 'eval-auc', mode = 'max').last_result

{'eval-auc': 0.859508,
 'time_this_iter_s': 0.035588741302490234,
 'done': False,
 'timesteps_total': 0,
 'episodes_total': 0,
 'training_iteration': 35,
 'trial_id': '193c3_00014',
 'experiment_id': '6942c984cda34128b94a4f338f24de98',
 'date': '2022-12-17_18-55-16',
 'timestamp': 1671303316,
 'time_total_s': 6.738027811050415,
 'pid': 7419,
 'hostname': '4a3396a947d5',
 'node_ip': '172.28.0.12',
 'config': {'objective': 'binary:logistic',
  'tree_method': 'gpu_hist',
  'eval_metric': ['auc'],
  'max_depth': 8,
  'min_child_weight': 4,
  'colsample_bytree': 0.5244859696763495,
  'eta': 0.08327688152347003,
  'reg_lambda': 2.796357221852045,
  'reg_alpha': 4.062567684689367,
  'n_estimators': 200,
  'seed': 0},
 'time_since_restore': 2.4659454822540283,
 'timesteps_since_restore': 0,
 'iterations_since_restore': 20,
 'warmup_time': 0.010481595993041992,
 'experiment_tag': '14_colsample_bytree=0.5245,eta=0.0833,max_depth=8,min_child_weight=4,reg_alpha=4.0626,reg_lambda=2.7964'}

### Helper Function

In [ ]:
### Helper function

def map_xgb(model, race = 'all', exclude = None, data = state_data_t):

    auc_all = pd.DataFrame({'state': [us_state_to_abbrev[s] for s in states], 'auc': np.zeros(len(states))})

    for i, state in enumerate(states):
        _, X_test, _, y_test = data[i]
        if(race != 'all'):
            if(race == 'black'):
                index = X_test['race4_Black or African American'] == 1
            if(race == 'hispanic'):
                index = X_test['race4_Hispanic/Latino'] == 1
            X_test = X_test.loc[index]
            y_test = y_test.loc[index]
        if(exclude != None):
            X_test = X_test.drop(columns = exclude)
        X_test = xgb.DMatrix(X_test.iloc[:,4:].values)
        y_pred = model.predict(X_test)
        fpr, tpr,_ = metrics.roc_curve(y_test, y_pred)
        auc = metrics.auc(fpr,tpr)
        auc_all.iloc[i,1] = auc

    fig = px.choropleth(auc_all,
                        locations='state', 
                        locationmode="USA-states", 
                        scope="usa",
                        color='auc',
                        color_continuous_scale="Viridis_r",
                        range_color = [0.6,1] 
                        )
    return fig, auc_all

### Model

In [ ]:
config =  {{'objective': 'binary:logistic',
  'tree_method': 'gpu_hist',
  'eval_metric': ['auc'],
  'max_depth': 10,
  'min_child_weight': 3,
  'colsample_bytree': 0.5020551988135382,
  'eta': 0.029222812929186416,
  'reg_lambda': 0.36756709262675025,
  'reg_alpha': 3.7137701473201687,
  'n_estimators': 200,
  'seed': 0}}

In [ ]:
train_set = xgb.DMatrix(X_train.iloc[:,4:].values,y_train.values)
val_set = xgb.DMatrix(X_val.iloc[:,4:].values,y_val.values)
results = {}
xgb_model = xgb.train(
    config,
    train_set,
    num_boost_round = 50,
    evals = [(val_set, 'eval')],
    evals_result = results,
    early_stopping_rounds=2,
)

[0]	eval-auc:0.78467
Will train until eval-auc hasn't improved in 2 rounds.
[1]	eval-auc:0.838631
[2]	eval-auc:0.846175
[3]	eval-auc:0.847867
[4]	eval-auc:0.84958
[5]	eval-auc:0.850606
[6]	eval-auc:0.850794
[7]	eval-auc:0.853739
[8]	eval-auc:0.854347
[9]	eval-auc:0.854317
[10]	eval-auc:0.855148
[11]	eval-auc:0.855674
[12]	eval-auc:0.856561
[13]	eval-auc:0.856937
[14]	eval-auc:0.856914
[15]	eval-auc:0.857663
[16]	eval-auc:0.857468
[17]	eval-auc:0.85742
Stopping. Best iteration:
[15]	eval-auc:0.857663



In [ ]:
xgb_model_map, xgb_auc = map_xgb(xgb_model)

In [ ]:
xgb_model_map

## Neural Network Model

### Finding the best hyperparameters & structure

In [ ]:
#Hyperparamter tuning

import tensorflow as tf
import math

from keras.models import Sequential
from keras.layers import Dense
from keras import initializers
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

from ray.tune.integration.keras import TuneReportCallback
from ray.tune.schedulers import HyperBandScheduler

def data_loader():
    return (X_train.iloc[:,4:].values,y_train.values), (X_val.iloc[:,4:].values,y_val.values)

def lr_step_decay(epoch, lr):
    drop_rate = 0.5
    epochs_drop = 5.0
    return 0.01 * math.pow(drop_rate, math.floor(epoch/epochs_drop))

def train_data(config,data):

    t1, t2 = data

    X_train = t1[0]
    y_train = t1[1]
    X_test = t2[0]
    y_test = t2[1]

    n_units = config['units']
    n_layers = config['layers']
    activation = config['activation']
    if(activation == 'tanh' or activation == 'sigmoid'):
        initializer = 'glorot_uniform'
    else:
        initializer = 'he_normal'

    model = Sequential()

    model.add(Dense(units = n_units, 
                    input_dim=X_train.shape[1], 
                    activation= activation,
                    kernel_initializer= initializer, 
                    name='h1'))
    
    for i in range(2, n_layers + 1):
        model.add(Dense(units= n_units, 
                        activation= activation,
                        kernel_initializer= initializer,  
                        name='h{}'.format(i)))
        
    model.add(Dense(units=1, activation='sigmoid', kernel_initializer=initializer, name='o'))

    model.compile(
        loss="binary_crossentropy", 
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=0.01), 
        metrics=['AUC'])

    model.fit(
        X_train,
        y_train,
        batch_size=128,
        epochs=50,
        verbose=0,
        validation_data=(X_test, y_test),
        callbacks=[TuneReportCallback({
            "mean_auc": "val_auc"
            }),
            LearningRateScheduler(
                lr_step_decay, verbose=0
            ),
            EarlyStopping(
                monitor='val_auc', 
                patience=6
            )
        ])


### GridSearch

In [ ]:
t1,t2 = data_loader()

config = {
          'units': tune.grid_search([10, 20, 40, 80]),
          'layers': tune.grid_search([2, 4, 6, 8, 10]),
          'activation': tune.grid_search(['ReLU','tanh','sigmoid'])
}

analysis = tune.run(
    tune.with_parameters(train_data, data = (t1,t2)),
    resources_per_trial = {"gpu":1},
    config = config,
    metric='mean_auc',
    mode="max"
)

2022-12-18 19:05:11,764	WARNING callback.py:108 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(train_data pid=8967) 2022-12-18 19:05:18.197280: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,mean_auc,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_data_e5a43_00000,2022-12-18_19-05-56,True,,60c9d603d72f45889ee39dd09cd87ab2,2a3ce8882f48,15,0.8614,172.28.0.12,8967,40.0499,2.44663,40.0499,1671390356,0,,15,e5a43_00000,0.005404
train_data_e5a43_00001,2022-12-18_19-06-39,True,,8677cd8bf24d49928178e012ac9c3169,2a3ce8882f48,14,0.841095,172.28.0.12,9144,37.5884,2.42114,37.5884,1671390399,0,,14,e5a43_00001,0.00417495
train_data_e5a43_00002,2022-12-18_19-07-18,True,,a69f018ea769474096668ea6cada5467,2a3ce8882f48,13,0.853887,172.28.0.12,9283,35.289,2.39763,35.289,1671390438,0,,13,e5a43_00002,0.00422978
train_data_e5a43_00003,2022-12-18_19-08-05,True,,be6401f0515d43cf94742224cc90cc2a,2a3ce8882f48,15,0.860292,172.28.0.12,9413,42.2984,2.56022,42.2984,1671390485,0,,15,e5a43_00003,0.00418353
train_data_e5a43_00004,2022-12-18_19-09-08,True,,cb39d4e75c044649af45a3f5451b08fd,2a3ce8882f48,22,0.851276,172.28.0.12,9560,58.2935,2.55806,58.2935,1671390548,0,,22,e5a43_00004,0.00383687
train_data_e5a43_00005,2022-12-18_19-10-08,True,,0eb0dda447914f8baf4df1a580c0a1f5,2a3ce8882f48,21,0.852697,172.28.0.12,9741,55.6408,2.48248,55.6408,1671390608,0,,21,e5a43_00005,0.00392604
train_data_e5a43_00006,2022-12-18_19-10-53,True,,f53eaadea7f14c8fa4bf9bcb7ff7f43b,2a3ce8882f48,14,0.861293,172.28.0.12,9919,40.3083,2.62383,40.3083,1671390653,0,,14,e5a43_00006,0.00385499
train_data_e5a43_00007,2022-12-18_19-11-28,True,,32b46669af854397bcc9a7c39f322ed2,2a3ce8882f48,10,0.786412,172.28.0.12,10061,29.8607,2.63024,29.8607,1671390688,0,,10,e5a43_00007,0.00397873
train_data_e5a43_00008,2022-12-18_19-12-23,True,,fcbb38cb74f34dacbd04f8c95d7de550,2a3ce8882f48,18,0.850712,172.28.0.12,10177,50.7175,2.63339,50.7175,1671390743,0,,18,e5a43_00008,0.00366735
train_data_e5a43_00009,2022-12-18_19-13-25,True,,c41d3e69f3e04534b751ae4cb76ac6cc,2a3ce8882f48,20,0.858103,172.28.0.12,10335,58.0467,2.7382,58.0467,1671390805,0,,20,e5a43_00009,0.0045712


(train_data pid=9144) 2022-12-18 19:06:03.103634: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_data pid=9283) 2022-12-18 19:06:44.972654: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_data pid=9413) 2022-12-18 19:07:24.888165: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_data pid=9560) 2022-12-18 19:08:11.930725: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_data pid=9741) 2022-12-18 19:09:14.832454

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,mean_auc,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_data_e5a43_00000,2022-12-18_19-05-56,True,,60c9d603d72f45889ee39dd09cd87ab2,"0_activation=ReLU,layers=2,units=10",2a3ce8882f48,15,0.8614,172.28.0.12,8967,40.0499,2.44663,40.0499,1671390356,0,,15,e5a43_00000,0.005404
train_data_e5a43_00001,2022-12-18_19-06-39,True,,8677cd8bf24d49928178e012ac9c3169,"1_activation=tanh,layers=2,units=10",2a3ce8882f48,14,0.841095,172.28.0.12,9144,37.5884,2.42114,37.5884,1671390399,0,,14,e5a43_00001,0.00417495
train_data_e5a43_00002,2022-12-18_19-07-18,True,,a69f018ea769474096668ea6cada5467,"2_activation=sigmoid,layers=2,units=10",2a3ce8882f48,13,0.853887,172.28.0.12,9283,35.289,2.39763,35.289,1671390438,0,,13,e5a43_00002,0.00422978
train_data_e5a43_00003,2022-12-18_19-08-05,True,,be6401f0515d43cf94742224cc90cc2a,"3_activation=ReLU,layers=4,units=10",2a3ce8882f48,15,0.860292,172.28.0.12,9413,42.2984,2.56022,42.2984,1671390485,0,,15,e5a43_00003,0.00418353
train_data_e5a43_00004,2022-12-18_19-09-08,True,,cb39d4e75c044649af45a3f5451b08fd,"4_activation=tanh,layers=4,units=10",2a3ce8882f48,22,0.851276,172.28.0.12,9560,58.2935,2.55806,58.2935,1671390548,0,,22,e5a43_00004,0.00383687
train_data_e5a43_00005,2022-12-18_19-10-08,True,,0eb0dda447914f8baf4df1a580c0a1f5,"5_activation=sigmoid,layers=4,units=10",2a3ce8882f48,21,0.852697,172.28.0.12,9741,55.6408,2.48248,55.6408,1671390608,0,,21,e5a43_00005,0.00392604
train_data_e5a43_00006,2022-12-18_19-10-53,True,,f53eaadea7f14c8fa4bf9bcb7ff7f43b,"6_activation=ReLU,layers=6,units=10",2a3ce8882f48,14,0.861293,172.28.0.12,9919,40.3083,2.62383,40.3083,1671390653,0,,14,e5a43_00006,0.00385499
train_data_e5a43_00007,2022-12-18_19-11-28,True,,32b46669af854397bcc9a7c39f322ed2,"7_activation=tanh,layers=6,units=10",2a3ce8882f48,10,0.786412,172.28.0.12,10061,29.8607,2.63024,29.8607,1671390688,0,,10,e5a43_00007,0.00397873
train_data_e5a43_00008,2022-12-18_19-12-23,True,,fcbb38cb74f34dacbd04f8c95d7de550,"8_activation=sigmoid,layers=6,units=10",2a3ce8882f48,18,0.850712,172.28.0.12,10177,50.7175,2.63339,50.7175,1671390743,0,,18,e5a43_00008,0.00366735
train_data_e5a43_00009,2022-12-18_19-13-25,True,,c41d3e69f3e04534b751ae4cb76ac6cc,"9_activation=ReLU,layers=8,units=10",2a3ce8882f48,20,0.858103,172.28.0.12,10335,58.0467,2.7382,58.0467,1671390805,0,,20,e5a43_00009,0.0045712


(train_data pid=14646) 2022-12-18 19:36:23.250766: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_data pid=14832) 2022-12-18 19:37:31.120975: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_data pid=14951) 2022-12-18 19:38:09.116656: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_data pid=15050) 2022-12-18 19:38:36.136352: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_data pid=15147) 2022-12-18 19:39:03.1

In [ ]:
analysis.get_best_trial(metric = 'mean_auc', mode = 'max').last_result

{'mean_auc': 0.8613995909690857,
 'time_this_iter_s': 2.4466311931610107,
 'done': True,
 'timesteps_total': None,
 'episodes_total': None,
 'training_iteration': 15,
 'trial_id': 'e5a43_00000',
 'experiment_id': '60c9d603d72f45889ee39dd09cd87ab2',
 'date': '2022-12-18_19-05-56',
 'timestamp': 1671390356,
 'time_total_s': 40.049864053726196,
 'pid': 8967,
 'hostname': '2a3ce8882f48',
 'node_ip': '172.28.0.12',
 'config': {'units': 10, 'layers': 2, 'activation': 'ReLU'},
 'time_since_restore': 40.049864053726196,
 'timesteps_since_restore': 0,
 'iterations_since_restore': 15,
 'warmup_time': 0.005403995513916016,
 'experiment_tag': '0_activation=ReLU,layers=2,units=10'}

### Hyperband

In [ ]:
t1,t2 = data_loader()

from ray.tune.schedulers import HyperBandScheduler

hyperband_scheduler = HyperBandScheduler(
    time_attr='training_iteration',
    metric='mean_auc',
    mode='max',
    max_t=50,
    reduction_factor=3)

config = {
          'units': tune.grid_search([10, 20, 40, 80]),
          'layers': tune.grid_search([2, 4, 6, 8, 10]),
          'activation': tune.grid_search(['ReLU','tanh','sigmoid'])
}

analysis_hyper = tune.run(
    tune.with_parameters(train_data, data = (t1,t2)),
    resources_per_trial = {"gpu":1},
    config = config,
    scheduler = hyperband_scheduler
)

2022-12-18 19:51:33,921	WARNING callback.py:108 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(train_data pid=17487) 2022-12-18 19:51:39.694724: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,mean_auc,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_data_5fefd_00000,2022-12-18_19-52-10,True,,980a254aca07454ead0b95aae5b1471d,2a3ce8882f48,12,0.857191,172.28.0.12,17487,31.805,2.36023,31.805,1671393130,0,,12,5fefd_00000,0.0083549
train_data_5fefd_00001,2022-12-18_19-54-29,True,,31b179bbfb334cbc9b2380ee3b6cbb86,2a3ce8882f48,8,0.841444,172.28.0.12,18048,22.3605,2.414,22.3605,1671393269,0,,8,5fefd_00001,0.00401974
train_data_5fefd_00002,2022-12-18_19-56-12,True,,cf12301ca7ca4099b4d68a9c0b6d4ea2,2a3ce8882f48,14,0.855485,172.28.0.12,18368,36.1497,2.33269,36.1497,1671393372,0,,14,5fefd_00002,0.00424957
train_data_5fefd_00003,2022-12-18_20-02-34,True,,c99b13163d1c4e82a712f60611317c6b,2a3ce8882f48,12,0.861092,172.28.0.12,20299,33.5437,2.49324,33.5437,1671393754,0,,12,5fefd_00003,0.00391412
train_data_5fefd_00004,2022-12-18_20-05-37,True,,4abde5f4b591418c90739d84df4e90c5,2a3ce8882f48,17,0.841895,172.28.0.12,21099,46.4751,2.52705,46.4751,1671393937,0,,17,5fefd_00004,0.00408053
train_data_5fefd_00005,2022-12-18_19-52-57,False,,2a259ba47c224235aca57379cf034d64,2a3ce8882f48,16,0.852815,172.28.0.12,17643,43.0248,2.43504,43.0248,1671393177,0,,16,5fefd_00005,0.00544858
train_data_5fefd_00006,2022-12-18_19-55-31,True,,ae4d2d894a6342648e1a941923d30ccf,2a3ce8882f48,13,0.857122,172.28.0.12,18236,38.043,2.64362,38.043,1671393331,0,,13,5fefd_00006,0.0040102
train_data_5fefd_00007,2022-12-18_19-58-03,False,,6373e2d531784631b24431011f842658,2a3ce8882f48,16,0.819013,172.28.0.12,18761,46.141,2.68394,46.141,1671393483,0,,16,5fefd_00007,0.00370717
train_data_5fefd_00008,2022-12-18_20-24-01,True,0,688be3cadd114e509802144209a478f4,2a3ce8882f48,11,0.841025,172.28.0.12,25138,34.5376,2.80139,82.6662,1671395041,0,0,11,5fefd_00008,0.00925779
train_data_5fefd_00009,2022-12-18_20-24-57,True,0,f52a9ae631654c49a77857b8af34dc9f,2a3ce8882f48,16,0.856134,172.28.0.12,25265,51.0819,2.84416,98.6824,1671395097,0,0,16,5fefd_00009,0.0127418


(train_data pid=17643) 2022-12-18 19:52:16.287652: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_data pid=17790) 2022-12-18 19:53:04.216563: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_data pid=17875) 2022-12-18 19:53:26.355689: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_data pid=17961) 2022-12-18 19:53:48.441846: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
(train_data pid=18048) 2022-12-18 19:54:08.3

In [ ]:
analysis_hyper.get_best_trial(metric = 'mean_auc', mode = 'max').last_result

{'mean_auc': 0.8610917329788208,
 'time_this_iter_s': 2.493243932723999,
 'done': True,
 'timesteps_total': None,
 'episodes_total': None,
 'training_iteration': 12,
 'trial_id': '5fefd_00003',
 'experiment_id': 'c99b13163d1c4e82a712f60611317c6b',
 'date': '2022-12-18_20-02-34',
 'timestamp': 1671393754,
 'time_total_s': 33.543747425079346,
 'pid': 20299,
 'hostname': '2a3ce8882f48',
 'node_ip': '172.28.0.12',
 'config': {'units': 10, 'layers': 4, 'activation': 'ReLU'},
 'time_since_restore': 33.543747425079346,
 'timesteps_since_restore': 0,
 'iterations_since_restore': 12,
 'warmup_time': 0.0039141178131103516,
 'experiment_tag': '3_activation=ReLU,layers=4,units=10'}

### Model